In [1]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [2]:
 #!pip install sqlalchemy

In [3]:
# db name is current schema
db_name = "university"
password="launchpass"

In [4]:
conn = pymysql.connect(host="127.0.0.1", port=3306, user="root", password=password, database=db_name);

In [5]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [6]:
tables = ["students",  "courses", "taken", "section", "department", "club", "in_club"]

for t in tables:
    df = pd.read_csv(t +  ".csv")
    df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)
    
    df.to_sql(name=t,  con=connection, if_exists='replace', index=True)

In [7]:
query1 = """ SELECT *
             from courses
             where course_title like "intro%%"; """

query2 = """ Select distinct students.student_id, first_name, students.dept_code
            from students
                join taken on students.student_id= taken.student_id
                join courses on courses.dept_code=students.dept_code;"""


query3= """Select first_name, last_name, total_credits
from students
where total_credits>=90;"""

query4= """select num_students, dept_name 
from department
order by num_students desc
limit 5;"""

query5="""select dept_name, budget
from department
order by budget desc
limit 5;"""


query6="""select count(course_id), year_offered 
from section
group by year_offered, semester;"""

query7="""select sum(num_registered) as total_registered, course_id  
from section
group by course_id
order by total_registered desc;"""



pd.read_sql(query2, con=connection)

,student_id,first_name,dept_code
0,1,John,ECON
1,2,Jon,BUS
2,3,Mary,NUTR
3,4,Mark,CS
4,6,Barbara,MUSI
5,7,Johnny,STAT
6,8,Jose,CS
7,9,Bo,CS
8,10,Forge,BIOL
9,11,Silas,STAT


In [8]:
fundQuery = """ SELECT *
from department;"""
df = pd.read_sql(fundQuery, con=connection)
df['Funding_Ratio']=df.num_students/df.num_staff
df.sort_values(by="Funding_Ratio", ascending=False)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [9]:
# students taking classes in their major
lastQuery= """select first_name, last_name, s.dept_code as student_dept, t.course_id, courses.dept_code as course_dept 
from students s join taken t on t.student_id = s.student_id
join courses on t.course_id=courses.course_id
where s.dept_code=courses.dept_code ;"""
pd.read_sql(lastQuery, con=connection)

,first_name,last_name,student_dept,course_id,course_dept
0,John,Doe,ECON,ECON125,ECON
1,Harold,Green,NUTR,NUTR201,NUTR
